In [1]:
from sklearn.datasets import fetch_20newsgroups
from matplotlib import pyplot as plt
from collections import Counter
newsgroups_train = fetch_20newsgroups(subset='train')
newsgroups_test = fetch_20newsgroups(subset='test')
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
import gensim
from gensim.models import KeyedVectors
import os, re, csv, math, codecs

In [2]:
token=Tokenizer(num_words=30000,
                filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n', lower=True,
                split=' ', char_level=False, oov_token="UNK", document_count=0)
token.fit_on_texts(newsgroups_train.data)
train_sequences=token.texts_to_sequences(newsgroups_train.data)
test_sequences=token.texts_to_sequences(newsgroups_test.data)
lengths=[len(seq) for seq in train_sequences]
lengths=dict(Counter(lengths))
max_len=500 #Incluso si algunos textos son mas largos, tener las primeras 500 palabras es suficiente en la clasificacion de textos
train_sequences=pad_sequences(train_sequences,maxlen=max_len)   #Las primeras palabras suelen contener toda la informacion nesesaria para la clasificacion
test_sequences=pad_sequences(test_sequences,maxlen=max_len)
reverse_dictionary = token.index_word
dictionary = dict([(value, key) for (key, value) in reverse_dictionary.items()])


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
!unzip 'drive/MyDrive/crawl-300d-2M.vec.zip'

Archive:  drive/MyDrive/crawl-300d-2M.vec.zip
  inflating: crawl-300d-2M.vec       


In [4]:
#Se utilizan los embeddigns de facebook pre-entrenados con Common Crawl
embeddings_index = KeyedVectors.load_word2vec_format('crawl-300d-2M.vec',
                                                     binary=False)

In [5]:
embed_dim=300
num_words=len(dictionary)+1
embedding_matrix=np.zeros([num_words,embed_dim])
for word, idx in dictionary.items():
  if idx <= num_words and word in embeddings_index:
    embedding_matrix[idx,:]=embeddings_index[word]

In [6]:
from tensorflow.keras.layers import Embedding, Conv1D, MaxPooling1D, MaxPooling1D, Dropout, Dense, Input, Concatenate,Dot,RepeatVector,TimeDistributed,Multiply,Lambda,Flatten, BatchNormalization, Activation, Bidirectional, LSTM, Reshape
import tensorflow.keras.backend as K
from tensorflow.keras.activations import softmax
from tensorflow.keras.models import Model
from tensorflow.keras import optimizers

# Red de attention + conv
### Parte convolucional:
Se utiliza una red convolucional luego de la capa de embeddings para optener una representacion de la palabra que incluya su contexto, es decir que a la salida se tienen vectores de dimencion 100 los cuales puede comprender un significado de la palabra en base a las palabras en su alrededor. La funcion tangente hyperbolica a la salida de la ultima capa convolucional se utiliza para escalar la salida. Se quiere valores pequeños a la salida para evitar problemas de baja gradiente en la funcion softmax.

### Mecanismo de attention
Dado que se quiere clasificar en distintas clases, en este caso necesitamos un mequanismo que sepa reconocer que palabras son relevantes para la clasificacion y cuales no. Por lo tanto, hay solo una preguntar para hacer (si la palabra es relevante o no) y esto lleva a que haya un solo vector de query, el cual se obtiene durante el entrenamiento de la capa.
Luego de obtener la representacion ponderada de cada palabra estas se suman para obtener la representacion ponderada del texto completo. No es necesario dividir el total por la cantidad de palabras ya que en el preprocesamiento se dejaron todos los vectores de textos con la misma dimencionalidad.



In [63]:
value_dim=100
nb_words=num_words
num_filters=64
input_layer = Input(shape=(max_len,))
embedding_layer=Embedding(nb_words, embed_dim, weights=[embedding_matrix], input_length=max_len, trainable=False)(input_layer)

#Capa convolucional
conv_out=Conv1D(value_dim,8,padding="same")(embedding_layer)
conv_out=Activation("relu")(conv_out)
conv_out=Conv1D(value_dim,8,activation="tanh",padding="same")(conv_out)

#Mecanismo de attention
#Producto punto y la salida hacia una softmax
ulog_attention=Dense(1,activation="softmax")(conv_out)
#obtengo los embeddings pesados
weighted_embeddings=Multiply()([ulog_attention,conv_out])
#Sumo las salidas para crear la representacion vectorial del texto
embedding_sum = Lambda(lambda x: K.sum(x, axis=1))(weighted_embeddings)

#MLP
dense1=Dense(100, activation='relu')(embedding_sum)
dense2=Dense(20, activation='softmax')(dense1)
model=Model(input_layer , dense2)

adam = optimizers.Adam(learning_rate=0.001)
model.compile(loss='sparse_categorical_crossentropy', optimizer=adam, metrics=['accuracy'])


In [64]:
model.summary()

Model: "model_18"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_22 (InputLayer)       [(None, 500)]                0         []                            
                                                                                                  
 embedding_21 (Embedding)    (None, 500, 300)             4024320   ['input_22[0][0]']            
                                                          0                                       
                                                                                                  
 conv1d_55 (Conv1D)          (None, 500, 100)             240100    ['embedding_21[0][0]']        
                                                                                                  
 activation_3 (Activation)   (None, 500, 100)             0         ['conv1d_55[0][0]']    

In [65]:
model.fit(train_sequences, newsgroups_train.target,batch_size=32,epochs=20,validation_split=0.2)

Epoch 1/20
283/283 [==============================] - 7s 17ms/step - loss: 3.7031 - accuracy: 0.4638 - val_loss: 1.1941 - val_accuracy: 0.6487
Epoch 2/20
283/283 [==============================] - 4s 15ms/step - loss: 0.9298 - accuracy: 0.7309 - val_loss: 0.7910 - val_accuracy: 0.7640
Epoch 3/20
283/283 [==============================] - 4s 16ms/step - loss: 0.6040 - accuracy: 0.8227 - val_loss: 0.6573 - val_accuracy: 0.8091
Epoch 4/20
283/283 [==============================] - 5s 16ms/step - loss: 0.4028 - accuracy: 0.8845 - val_loss: 0.7606 - val_accuracy: 0.8016
Epoch 5/20
283/283 [==============================] - 4s 15ms/step - loss: 0.3277 - accuracy: 0.9059 - val_loss: 0.6420 - val_accuracy: 0.8272
Epoch 6/20
283/283 [==============================] - 4s 15ms/step - loss: 0.2558 - accuracy: 0.9292 - val_loss: 0.6380 - val_accuracy: 0.8471
Epoch 7/20
283/283 [==============================] - 176s 624ms/step - loss: 0.1703 - accuracy: 0.9508 - val_loss: 0.6199 - val_accuracy: 0.8

## Red Attention + Bidirectional LSTM

Se reemplaza la parte convolucional de la red con una capa LSTM Bidireccional para obtener otro tipo de representacion de cada palabra con el contexto del resto del texto. Dada la bidireccionalidad se obtiene contexto para la palabra desde las palabras anteriores y las posteriores.

In [8]:
value_dim=100
nb_words=num_words
num_filters=64
input_layer = Input(shape=(max_len,))
embedding_layer=Embedding(nb_words, embed_dim, weights=[embedding_matrix], input_length=max_len, trainable=False)(input_layer)

#Bidirectional LSTM
lstm_out=Bidirectional(LSTM(value_dim, return_sequences=True,activation="tanh", unroll=True), merge_mode="mul")(embedding_layer)

def softMaxOverTime(x):
    return softmax(x,axis=1)

#Mecanismo de attention

ulog_attention=Dense(1,activation="linear")(lstm_out)
attention=Activation(softMaxOverTime)(ulog_attention)
repeated_attention=TimeDistributed(RepeatVector(value_dim))(attention)
repeated_attention=Reshape([max_len,value_dim])(repeated_attention)
weighted_embeddings=Multiply()([repeated_attention,lstm_out])
embedding_sum = Lambda(lambda x: K.sum(x, axis=1))(weighted_embeddings)

#MLP
dense1=Dense(100, activation='relu')(embedding_sum)
dense2=Dense(20, activation='softmax')(dense1)
model=Model(input_layer , dense2)

adam = optimizers.Adam(learning_rate=0.001)
model.compile(loss='sparse_categorical_crossentropy', optimizer=adam, metrics=['accuracy'])


In [9]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, 500)]                0         []                            
                                                                                                  
 embedding_1 (Embedding)     (None, 500, 300)             4024320   ['input_2[0][0]']             
                                                          0                                       
                                                                                                  
 bidirectional (Bidirection  (None, 500, 100)             320800    ['embedding_1[0][0]']         
 al)                                                                                              
                                                                                              

In [10]:
model.fit(train_sequences, newsgroups_train.target,batch_size=516,epochs=40,validation_split=0.2)

Epoch 1/40
18/18 [==============================] - 280s 2s/step - loss: 2.9725 - accuracy: 0.0963 - val_loss: 2.8533 - val_accuracy: 0.1255
Epoch 2/40
18/18 [==============================] - 8s 445ms/step - loss: 2.6864 - accuracy: 0.1463 - val_loss: 2.4579 - val_accuracy: 0.2117
Epoch 3/40
18/18 [==============================] - 7s 385ms/step - loss: 2.2540 - accuracy: 0.2569 - val_loss: 2.1327 - val_accuracy: 0.2581
Epoch 4/40
18/18 [==============================] - 8s 473ms/step - loss: 1.9853 - accuracy: 0.3305 - val_loss: 1.8137 - val_accuracy: 0.4114
Epoch 5/40
18/18 [==============================] - 8s 474ms/step - loss: 1.6846 - accuracy: 0.4235 - val_loss: 1.5677 - val_accuracy: 0.4490
Epoch 6/40
18/18 [==============================] - 9s 532ms/step - loss: 1.4781 - accuracy: 0.4882 - val_loss: 1.4408 - val_accuracy: 0.4932
Epoch 7/40
18/18 [==============================] - 9s 493ms/step - loss: 1.3161 - accuracy: 0.5360 - val_loss: 1.2576 - val_accuracy: 0.5647
Epoch 8

Se puede finalizar observando que el uso de convoluciones vuelve el entrenamiento mas rapido aunque la red tienede a overfitting. en el caso del LSTM bidireccional, esta es de entrenamiento mas lento, incluso con el unroll=True, aunque se pueden encontrar mejores resultados.